# Dependences

In [2]:
using CUDA
using DelimitedFiles
using Test
using BenchmarkTools

# initial data
R_Agg = 13
r_max = 3.0
n_knn = 50
idx_red_size =  r_max ≤ 2.80 ? 13 :
            2.80 < r_max ≤ 3.45 ? 21 :
            3.45 < r_max ≤ 3.80 ? 39 :
            3.80 < r_max ≤ 4.00 ? 55 :
            70
X = Float64.(readdlm("../../../data/init/Sphere/$(R_Agg).xyz")[3:end,2:end]) |> cu

# kNN
idx      = Int32.(zeros(size(X, 1), size(X, 1))) |> cu;
idx_sum  = Int32.(zeros(1, size(idx, 1))) |> cu
idx_red  = Int32.(zeros(idx_red_size, size(idx, 1))) |> cu
idx_contractile =Int32.(zeros(n_knn,size(X,1))) |> cu

# forces
force = CUDA.zeros(size(X));

# Generic Functions

In [3]:
# force_func
cubic(A,r_max,r_min,dist) = - A * (dist-r_max)^2* (dist-r_min)

# distances
euclidean(points,i,j) = sqrt((points[i,1]-points[j,1])^2+(points[i,2]-points[j,2])^2+(points[i,3]-points[j,3])^2)

euclidean (generic function with 1 method)

# idx

## functions

In [4]:
function dist_kernel!(idx, points ,r_max)
    # Defining Index for kernel
    i = (blockIdx().x - 1) * blockDim().x + threadIdx().x
    j = (blockIdx().y - 1) * blockDim().y + threadIdx().y
    
    # Limiting data inside matrix
    if i <= size(points, 1) && j <= size(points, 1)
        if euclidean(points,i,j) < r_max
            idx[i, j] = i
        else
            idx[i, j] = 0
        end 
    end
    return nothing
end

function reduce_kernel(idx,idx_red,idx_sum)
    # Defining Index for kernel
    i  = (blockIdx().x-1) * blockDim().x + threadIdx().x

    # Limiting data inside matrix
    if i <= size(idx,1)
        # Cleaning idx_sum
        idx_sum[i] = 0
        
        # looping on each row for searching non-zero values
        for j = 1:size(idx,1)
            if idx[j,i] != 0
                idx_sum[i] += 1
                idx_red[idx_sum[i],i] = j
            end
        end
    end
    
    return nothing
end

function index_contractile!(idx_contractile,idx_sum,idx_red)
    # Defining Index for kernel
    i = (blockIdx().x - 1) * blockDim().x + threadIdx().x
    j = (blockIdx().y - 1) * blockDim().y + threadIdx().y

    # Limiting data inside matrix
    if i <= size(idx_contractile, 1) && j <= size(idx_contractile,2)
        idx_contractile[i,j] = idx_red[rand(1:idx_sum[j]),j]
    end
    return nothing
end

function nearest_neighbors(idx, idx_red, idx_sum, idx_contractile, points ,r_max)
    # Calculating Distance Matrix
    threads =(32,32)
    blocks  =cld.(size(points,1),threads)
    CUDA.@time @cuda threads=threads blocks=blocks dist_kernel!(idx, points ,r_max)

    # Reducing Distance Matrix to Nearest Neighbors
    threads=1024
    blocks=cld.(size(idx,1),threads)
    CUDA.@time @cuda threads=threads blocks=blocks reduce_kernel(idx,idx_red,idx_sum)

    # Finding index contractile
    threads =(32,32)
    blocks  =cld.(size(X,1),threads)
    CUDA.@time @cuda threads=threads blocks=blocks index_contractile!(idx_contractile,idx_sum,idx_red)
end

nearest_neighbors (generic function with 1 method)

## running

In [14]:
println("---------------------------------- VARIABLES ------------------------------------")
println("R_Agg = $(R_Agg) | R_Max = $(r_max) | col_size_idx = $(idx_red_size)")
display(X)
println("------------------------------------ SIZES --------------------------------------")
println("Size → idx     = $(size(idx))")
println("     → idx_sum = $(size(idx_sum))")
println("     → idx_red = $(size(idx_red))")
println("------------------------------------ RESULTS ------------------------------------")
nearest_neighbors(idx, idx_red, idx_sum,idx_contractile, X ,r_max)
println("----------------------------------- IDX RED -------------------------------------")
display(idx_red)
println("----------------------------------- IDX SUM -------------------------------------")
display(idx_sum)
println("------------------------------- IDX CONTRACTILE ---------------------------------")
display(idx_contractile)
println("---------------------------------- Memory Used ----------------------------------")
CUDA.memory_status()  

---------------------------------- VARIABLES ------------------------------------
R_Agg = 13 | R_Max = 3.0 | col_size_idx = 21


1620×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 -0.5  -4.04  -12.25
  1.5  -4.04  -12.25
 -3.5  -2.31  -12.25
 -1.5  -2.31  -12.25
  0.5  -2.31  -12.25
  2.5  -2.31  -12.25
 -2.5  -0.58  -12.25
 -0.5  -0.58  -12.25
  1.5  -0.58  -12.25
  3.5  -0.58  -12.25
 -3.5   1.15  -12.25
 -1.5   1.15  -12.25
  0.5   1.15  -12.25
  ⋮           
  0.5  -1.15   12.25
  2.5  -1.15   12.25
 -2.5   0.58   12.25
 -0.5   0.58   12.25
  1.5   0.58   12.25
  3.5   0.58   12.25
 -3.5   2.31   12.25
 -1.5   2.31   12.25
  0.5   2.31   12.25
  2.5   2.31   12.25
 -0.5   4.04   12.25
  1.5   4.04   12.25

------------------------------------ SIZES --------------------------------------
Size → idx     = (1620, 1620)
     → idx_sum = (1, 1620)
     → idx_red = (21, 1620)
------------------------------------ RESULTS ------------------------------------
  0.002962 seconds (69 CPU allocations: 4.016 KiB)
  0.001952 seconds (68 CPU allocations: 4.125 KiB)
  0.000152 seconds (29 CPU allocations: 1.625 KiB)
----------------------------------- IDX RED -------------------------------------


21×1620 CuArray{Int32, 2, CUDA.Mem.DeviceBuffer}:
  1   1   3   1   1   2   3   4   5  …  1580  1581  1582  1583  1589  1590
  2   2   4   3   2   5   4   5   6     1581  1582  1583  1584  1590  1591
  4   5   7   4   4   6   7   7   8     1582  1583  1584  1585  1591  1592
  5   6  23   5   5   9   8   8   9     1588  1589  1590  1591  1596  1597
 19  20  29   7   6  10  11   9  10     1589  1590  1591  1592  1597  1598
 24  25  30   8   8  26  12  12  13  …  1595  1596  1597  1598  1601  1602
 25  26  35  24   9  32  30  13  14     1611  1611  1612  1613  1616  1617
 30  31  36  30  25  33  36  31  32     1615  1612  1613  1614  1617  1618
 31  32  37  31  31  38  37  37  38     1616  1615  1616  1617  1619  1619
 32  33   0  36  32  39  43  38  39        0  1616  1617  1618  1620  1620
  0   0   0  37  37  40  44  44  45  …     0  1617  1618  1620     0     0
  0   0   0  38  38   0  45  45  46        0  1619  1619     0     0     0
  0   0   0   0  39   0   0  46  47        0     0

----------------------------------- IDX SUM -------------------------------------


1×1620 CuArray{Int32, 2, CUDA.Mem.DeviceBuffer}:
 10  10  9  12  13  11  12  13  13  10  …  13  13  10  9  12  13  11  10  10

------------------------------- IDX CONTRACTILE ---------------------------------


50×1620 CuArray{Int32, 2, CUDA.Mem.DeviceBuffer}:
 30  31  30  30  32  10  11  44   5  …  1595  1596  1619  1613  1589  1620
 24  26  37   1  39  38  12   9   5     1611  1596  1582  1614  1591  1592
 24  33   4   1   9  38  44  37  14     1588  1596  1582  1614  1616  1620
 25   1   3   5   6  33   8  31  46     1615  1616  1582  1598  1620  1620
  5  25  23  30  38  38  43  44  10     1589  1582  1583  1584  1596  1617
 30   5  36   1   1   2  30   9  45  …  1589  1581  1597  1585  1616  1602
 30   6  29  30   9   5   4  12  32     1616  1583  1616  1584  1620  1618
 25   5  29   4  37  39  30   9  39     1611  1617  1619  1591  1591  1602
 32   5  30   4  37  26  12   7  14     1611  1612  1617  1620  1591  1618
  4  33   7   3   4  38  37  13  45     1589  1581  1618  1613  1596  1590
 25  31  30  38  31  33  43  12  45  …  1616  1612  1612  1584  1589  1617
  5   6   7  24  32  40   8  45   8     1611  1583  1591  1613  1617  1590
 24   2  29   8  37  26  30   8  14     1582  1596

---------------------------------- Memory Used ----------------------------------
Effective GPU memory usage: 18.07% (361.875 MiB/1.955 GiB)
No memory pool is in use.

# forces

## functions

In [6]:
function sum_force!(idx,points,force, force_func,A,r_max,r_min)
    # Defining Index for kernel
    i = (blockIdx().x - 1) * blockDim().x + threadIdx().x
    k = (blockIdx().y - 1) * blockDim().y + threadIdx().y

    # Limiting data inside matrix
    if i <= size(points, 1) && k <= size(points, 2)

        # Cleaning idx_sum
        force[i,k] = 0
        dist = 0

        # Iterate on each row
        for j=1:size(idx,1)

            # Finding forces
            if idx[j,i] != i && idx[j,i] != 0
                dist = euclidean(points,i,idx[j,i])
                force[i,k] +=force_func(A,r_max,r_min,dist) * points[idx[j,i],k] / dist
            end
            
        end

        # PENDING
        # force[i,k] += idx[rand(1:idx_sum[k]),k]
        
    end
    return nothing
end

sum_force! (generic function with 1 method)

## running

In [53]:
A     = 1
r_min = 2.0
r_max = r_max

# GPU
threads =(200,3)
blocks  =cld.(size(X,1),threads)
CUDA.@time @cuda threads=threads blocks=blocks sum_force!(idx_red, X ,force, cubic,A,r_max,r_min)

# CPU
X_CPU = Matrix(X)
idx_red_CPU = Matrix(idx_red)
force_CPU = zeros(size((X)))

CUDA.@time for j = 1:size(idx_red_CPU,1)
    for i =1:size(X,1)
        if idx_red_CPU[j,i] != 0 && idx_red_CPU[j,i] != i
            dist = euclidean(X_CPU,i,idx_red_CPU[j,i])
            force_CPU[i,:] += cubic.(A,r_max,r_min,dist) .* X_CPU[idx_red_CPU[j,i],:] ./ dist
        end
    end
end

# testing
@test Matrix(force) ≈ force_CPU atol=0.01

  0.000956 seconds (84 CPU allocations: 4.875 KiB)
  0.089579 seconds (612.75 k CPU allocations: 24.167 MiB, 8.55% gc time)


Test Passed

In [60]:
CUDA.@time @cuda threads=threads blocks=blocks sum_force!(idx_red, X ,force, cubic,A,r_max,r_min)

  0.000965 seconds (84 CPU allocations: 4.875 KiB)


CUDA.HostKernel{typeof(sum_force!), Tuple{CuDeviceMatrix{Int32, 1}, CuDeviceMatrix{Float32, 1}, CuDeviceMatrix{Float32, 1}, typeof(cubic), Int64, Float64, Float64}}(sum_force!, CuFunction(Ptr{CUDA.CUfunc_st} @0x0000000007faabf0, CuModule(Ptr{CUDA.CUmod_st} @0x000000000817b520, CuContext(0x0000000004be0280, instance 5a68b831b99de127))), CUDA.KernelState(Ptr{Nothing} @0x0000000203400000))